类似\__slots\__、\__len\__()形如\__xxx\__的变量或者函数名作为特殊用途。帮助我们定制类。

\__str\__，\__repr\__

In [1]:
class Student(object):
    def __init__(self, name):
        self.name = name
        
print(Student('Michael'))

In [3]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name: %s)' % self.name

print(Student('Michael')) #调用的是__str__()，返回用户看到的字符串
Student('Michael') #调用的是__repr__()，返回程序开发者看到的字符串

Student object (name: Michael)


In [4]:
class Student(object):
    def __init__(self, name):
        self.name = name
    def __str__(self):
        return 'Student object (name=%s)' % self.name
    __repr__ = __str__

print(Student('Michael')) 
Student('Michael') 

Student object (name=Michael)


Student object (name=Michael)

\__iter\__

如果一个类想被用于for ... in循环，类似list或tuple那样，就必须实现一个__iter__()方法，该方法返回一个迭代对象，然后，Python的for循环就会不断调用该迭代对象的__next__()方法拿到循环的下一个值，直到遇到StopIteration错误时退出循环。

In [6]:
class Fib(object):
    def __init__(self):
        self.a, self.b = 0, 1 # 初始化两个计数器a，b

    def __iter__(self):
        return self # 实例本身就是迭代对象，故返回自己

    def __next__(self):
        self.a, self.b = self.b, self.a + self.b # 计算下一个值
        if self.a > 10: # 退出循环的条件
            raise StopIteration()
        return self.a # 返回下一个值

for n in Fib():
    print(n)

1
1
2
3
5
8


\__getitem\__

Fib实例虽然能作用于for循环，看起来和list有点像，但是，把它当成list来使用还是不行，比如，不能取第5个元素Fib()[5]。

要表现得像list那样按照下标取出元素，需要实现\__getitem__()方法。

In [ ]:
class Fib(object):
    def __getitem__(self, n):
        a, b = 1, 1
        for x in range(n):
            a, b = b, a + b
        return a

实现切片方法：\__getitem__()传入的参数可能是一个int，也可能是一个切片对象slice，所以要做判断

In [7]:
class Fib(object):
    def __getitem__(self, n):
        if isinstance(n, int): # n是索引
            a, b = 1, 1
            for x in range(n):
                a, b = b, a + b
            return a
        if isinstance(n, slice): # n是切片
            start = n.start
            stop = n.stop
            if start is None:
                start = 0
            a, b = 1, 1
            L = []
            for x in range(stop):
                if x >= start:
                    L.append(a)
                a, b = b, a + b
            return L

f = Fib()
f[0:5]
#但是没有对step参数作处理，也没有对负数作处理
#此外，如果把对象看成dict，__getitem__()的参数也可能是一个可以作key的object，例如str
#与之对应的是__setitem__()方法，把对象视作list或dict来对集合赋值。最后，还有一个__delitem__()方法，用于删除某个元素。


[1, 1, 2, 3, 5]

\__getattr\__

我们调用类的方法或属性时，如果不存在，就会报错。

除了可以加上一个score属性外，可用\__getattr\__()方法，动态返回一个属性。

只有在没有找到属性的情况下，才调用\__getattr\__，已有的属性不会在\__getattr\__中查找

In [12]:
class Student(object):

    def __init__(self):
        self.name = 'Michael'

    def __getattr__(self, attr):
        if attr=='score':
            return 99

s = Student()
print(s.score)
print(s.abc)  # __getattr__默认返回是None

99
None


In [14]:
class Student(object):

    def __getattr__(self, attr):
        if attr=='age':
            return lambda: 25  # 返回函数
        raise AttributeError('\'Student\' object has no attribute \'%s\'' % attr)
        #只响应特定的几个属性，其他抛出AttributeError的错误：
s = Student()
print(s.age())
print(s.abc)

25


AttributeError: 'Student' object has no attribute 'abc'

这实际上可以把一个类的所有属性和方法调用全部动态化处理了，不需要任何特殊手段。

完全动态调用的特性的作用：可以针对完全动态的情况作调用。

例如：现在很多网站都搞REST API，调用API的URL类似。
如果要写SDK，给每个URL对应的API都写一个方法，那得累死，而且，API一旦改动，SDK也要改。

利用完全动态的\__getattr\__，可以写一个链式调用：

In [15]:
class Chain(object):

    def __init__(self, path=''):
        self._path = path

    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))

    def __str__(self):
        return self._path

    __repr__ = __str__
    
Chain().status.user.timeline.list
#这样，无论API怎么变，SDK都可以根据URL实现完全动态的调用，而且，不随API的增加而改变！

/status/user/timeline/list

还有些REST API会把参数放到URL中，比如GitHub的API：

GET /users/:user/repos

调用时，需要把:user替换为实际用户名。

In [20]:
class Chain(object):

    def __init__(self, path=''):
        self._path = path

    def __getattr__(self, path):
        return Chain('%s/%s' % (self._path, path))

    def __str__(self):
        return self._path

    __repr__ = __str__
    __call__ = __getattr__
    
Chain().users('michael').repos
#Chain().users：没有实例对象属性users，所以调用__getattr__并返回一个新的对象实例s = Chain().users
#下一步Chain().users('michael')属于对新实例的直接调用，即s('michael')
#__call__ = __getattr__
#所以s('michael')相当于s.__call__('michael')，返回的新对象实例继续调用repos属性

/users/michael/repos

\__call\__

调用实例方法时，用instance.method()。

还可以直接在实例本身上调用。只需要定义一个\__call\__()方法，就可以直接对实例进行调用

In [22]:
class Student(object):
    def __init__(self, name):
        self.name = name

    def __call__(self):
        print('My name is %s.' % self.name)

s = Student('Tian Tian')
s()

My name is Tian Tian.


\__call\__()还可以定义参数。对实例进行直接调用就好比对一个函数进行调用一样，所以你完全可以把对象看成函数，把函数看成对象，因为这两者之间本来就没啥根本的区别。

如果你把对象看成函数，那么函数本身其实也可以在运行期动态创建出来，因为类的实例都是运行期创建出来的，这么一来，我们就模糊了对象和函数的界限。

那么，怎么判断一个变量是对象还是函数呢？其实，更多的时候，我们需要判断一个对象是否能被调用，能被调用的对象就是一个Callable对象，比如函数和我们上面定义的带有\__call\__()的类实例：

In [25]:
#通过callable()函数，我们就可以判断一个对象是否是“可调用”对象。
print(callable(Student('TT')))
print(callable([1, 2, 3]))

True
False
